In [1]:
from keras.optimizers import SGD
from keras.preprocessing.image import ImageDataGenerator

C:\Users\I353565\AppData\Local\Anaconda\Continuum\anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


#### 1. Load the pretrained model

In [2]:
from keras.applications import VGG16

vgg_model = VGG16(weights='imagenet', include_top = False, input_shape=(224,224,3))

In [3]:
train_datagen = ImageDataGenerator(
      rescale=1./255,
      rotation_range=20,
      width_shift_range=0.2,
      height_shift_range=0.2,
      horizontal_flip=True,
      fill_mode='nearest')
 
validation_datagen = ImageDataGenerator(rescale=1./255)
 
# Change the batchsize according to your system RAM
train_batchsize = 50
val_batchsize = 10
image_size = 224
 
train_generator = train_datagen.flow_from_directory(
        '../Data/Train',
        target_size=(image_size, image_size),
        batch_size=train_batchsize,
        class_mode='categorical')
 
validation_generator = validation_datagen.flow_from_directory(
        '../Data/Train',
        target_size=(image_size, image_size),
        batch_size=val_batchsize,
        class_mode='categorical',
        shuffle=False)

Found 2002 images belonging to 2 classes.
Found 2002 images belonging to 2 classes.


In [4]:
# I'm freezing the layers of vgg_net(all the conv layers except the last 4[3 Conv + 1 Pooling])
for layer in vgg_model.layers[:-4]:
    layer.trainable = False

# Check the trainable status of the individual layers
for layer in vgg_model.layers:
    print(layer, layer.trainable)


<keras.engine.input_layer.InputLayer object at 0x0000023DA87A87B8> False
<keras.layers.convolutional.Conv2D object at 0x0000023DA87A8C88> False
<keras.layers.convolutional.Conv2D object at 0x0000023DA87A8B00> False
<keras.layers.pooling.MaxPooling2D object at 0x0000023DA87CD048> False
<keras.layers.convolutional.Conv2D object at 0x0000023DA87CDE80> False
<keras.layers.convolutional.Conv2D object at 0x0000023DA880A0F0> False
<keras.layers.pooling.MaxPooling2D object at 0x0000023DA88275F8> False
<keras.layers.convolutional.Conv2D object at 0x0000023DA8827C50> False
<keras.layers.convolutional.Conv2D object at 0x0000023DA885E1D0> False
<keras.layers.convolutional.Conv2D object at 0x0000023DA8875080> False
<keras.layers.pooling.MaxPooling2D object at 0x0000023DA8891358> False
<keras.layers.convolutional.Conv2D object at 0x0000023DA88919B0> False
<keras.layers.convolutional.Conv2D object at 0x0000023DA88B1898> False
<keras.layers.convolutional.Conv2D object at 0x0000023DA88CB208> False
<ker

The actual VGGNet has the last layer with 1000 channels (1 for each class). I'll replace that with 2 and then try to fine tune the model

In [5]:
from keras import models
from keras import layers
from keras import optimizers
 
# Create the model
model = models.Sequential()
 
# Add the vgg convolutional base model
model.add(vgg_model)
 
# Add new layers
model.add(layers.Flatten())
model.add(layers.Dense(4096, activation='relu'))
model.add(layers.Dense(4096, activation='relu'))
model.add(layers.Dense(2, activation='softmax'))
 
# Show a summary of the model. Check the number of trainable parameters
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 7, 7, 512)         14714688  
_________________________________________________________________
flatten_1 (Flatten)          (None, 25088)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 4096)              102764544 
_________________________________________________________________
dense_2 (Dense)              (None, 4096)              16781312  
_________________________________________________________________
dense_3 (Dense)              (None, 2)                 8194      
Total params: 134,268,738
Trainable params: 126,633,474
Non-trainable params: 7,635,264
_________________________________________________________________


In [ ]:
# Compile the model
model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.RMSprop(lr=1e-4),
              metrics=['acc'])
# Train the model
history = model.fit_generator(
      train_generator,
      steps_per_epoch=train_generator.samples/train_generator.batch_size ,
      epochs=10,
      validation_data=validation_generator,
      validation_steps=validation_generator.samples/validation_generator.batch_size,
      verbose=1)
 
# Save the model
model.save('small_last4.h5')

Epoch 1/10
23/40 [================>.............] - ETA: 12:21 - loss: 7.8672 - acc: 0.4878